In [3]:
import music21 as m21
import re
from collections import Counter
from swalign_local import *

In [4]:
# Step 1: Read input data:
import music21 as m21
import re
from collections import Counter
from swalign_local import *
import csv
import glob

score_path = "jkupddtest/bachmono/wtc2f20.krn"
#score_path = "jkupddtest/beethovenmono/sonata01-3.krn"
#score_path = "jkupddtest/mozartmono/sonata04-2.krn"
#score_path = "jkupddtest/gibbonsmono/silverswan.krn"
#score_path = "jkupddtest/chopinmono/mazurka24-4.krn"
#score_path = '~/facets-search-engine/data/Beethoven9thOdeToJoy.xml'

raw_score = m21.converter.parse(score_path)
num_of_parts = len(raw_score.parts)

In [5]:
num_of_parts

3

In [6]:
voices = []
for part in raw_score.parts:
    voices.append(part)

In [7]:
# Step 2: Get diatonic scale degree sequence from all voices

def get_diatonic_intervals(voice):
    
    # get key / root information:
    roots = [key.tonic for key in voice.recurse().getElementsByClass(m21.key.Key)]
    diatonic_root = roots[0].diatonicNoteNum if roots != [] else voice.analyze('key').tonic.diatonicNoteNum
    #print(diatonic_root)
    key = voice.analyze('key')

    noteoffset = []
    midi = []
    #timesig = m21_score.getContextByClass(m21.meter.TimeSignature)
    #barlength = timesig.barduration

    dict_wordtonum = {"Unison": '0', "Second": '1', "Third": '2', "Fourth": '3', "Fifth": '4', "Sixth": '5', "Seventh": '6'}
        
    dict_encode_dia_intervals = {"6D": 'A', '5D': 'B', '4D': 'C', '3D':'D', '2D':'E', '1D':'F', 
                                     '1A':'G', '2A':'H', '3A':'I', '4A':'J', '5A': 'K', '6A':'L', '0A': 'M'}
        
    diatonic_intervals = []
    beatstrength = []
    previous_note = None

    onset = 0
    for thisnote in voice.recurse().notes:
        onset += thisnote.offset
        
        # We ignore rests
        if thisnote.isRest: 
                # If the rest is a full measure, part of a multi-measure rest: we need to adjust
                continue
        
        noteoffset.append(onset)
        beatstrength.append(thisnote.beatStrength)
        #if thisnote.isNote:
        #    midi.append(thisnote.pitch.midi)
        #elif thisnote.isChord:
        #    midi.append(thisnote.root().midi)
            
        if previous_note is None:
                previous_note = thisnote
        else:
                if thisnote.isNote and previous_note.isNote:
                    # gap = number of semi-tones of the current interval 
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isNote:
                    gap = thisnote.root().diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isChord:
                    gap = thisnote.root().diatonicNoteNum - previous_note.root().diatonicNoteNum
                elif thisnote.isNote and previous_note.isChord:
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.root().diatonicNoteNum
                # if a pitch change is detected
                    
                if gap != 0:
                    
                    if gap > 0:
                        #  if the semi-tone difference between the current and the previous item > 0, it is an ascending interval.
                        direction = 'A'
                    else:
                        #  otherwise, it is a descending interval.
                        direction = 'D'

                    # Get intervals using music21
                    """
                            "directedSimpleNiceName" examples: "Descending Doubly-Diminished Fifth", "Ascending Perfect Fourth", "Ascending Doubly-Augmented Fourth"
                            "simpleName" examples: dd5, P5, AA4. There's no direction information
                            Since it only executes when a pitch interval is detected, "unison" refers to an augmented unison, a.k.a minor second
                    """
                    # take intervals between root notes if there exists any chord
                    if previous_note.isChord:
                        startnote = previous_note.root()
                    else:
                        startnote = previous_note
                    if thisnote.isChord:
                        endnote = thisnote.root()
                    else:
                        endnote = thisnote
                        
                    m21_interval_directed = m21.interval.Interval(noteStart=startnote, noteEnd=endnote).directedSimpleNiceName

                    arr_diatonic = m21_interval_directed.split(" ")

                    m21_generic = dict_wordtonum[arr_diatonic[-1]]
                    
                    # m21_interval: 2A, 3D etc...
                    m21_interval = m21_generic+direction
                    if m21_generic == "0":
                        m21_interval = '0A'
                    # to make each m21_interval unique, show direction and each as single character in string, we encode the diatonic intervals as letters
                    encode_interval = dict_encode_dia_intervals[m21_interval]
                    diatonic_intervals.append(encode_interval)

                else:
                    # We take the interval between two consecutive pitches as 0A
                    encode_interval = dict_encode_dia_intervals['0A']
                    diatonic_intervals.append(encode_interval)
                previous_note = thisnote
            
    string = ""
    for i in diatonic_intervals:
        string += str(i)
        
    return string, key, noteoffset, midi, beatstrength

In [8]:
strings = []
noteoffsets = []
midis = []
keys = []
beatstrengths = []

for voice in voices:
    string, key, noteoffset, midi, beatstrength = get_diatonic_intervals(voice)
    strings.append(string)
    noteoffsets.append(noteoffset)
    beatstrengths.append(beatstrength)
    midis.append(midi)
    keys.append(key)
    print(len(string), len(beatstrength))#, len(midi))
    print(string)
    print(len(noteoffset))
    #print(midi)
    print(beatstrength)
    

210 211
EIAJEIEEEKEEEKEGGMMGGGFFFFGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGDEIEMGMFGGGGGGGGGFIAJEIEEEKEEGGGFFFFHGGGGFFFFFFFGFFFFFFFMFMCIMMIMHCHCHMIFFFFHHHFEFFGGCLFIEIAJEIEEHEIEEEKEEEKEMFGGGGGGGFFFFFFFIFFFFFFFIFFFFFFFKFGGMFGGMGCHG
211
[0.25, 0.5, 0.25, 1.0, 0.125, 0.25, 0.125, 1.0, 0.125, 0.25, 0.125, 0.5, 0.125, 0.25, 0.125, 1.0, 0.125, 0.25, 0.5, 0.0625, 0.0625, 0.0625, 0.125, 0.0625, 0.0625, 0.0625, 0.25, 0.125, 0.0625, 0.0625, 0.0625, 0.25, 0.125, 0.0625, 0.0625, 0.0625, 0.5, 0.0625, 0.0625, 0.125, 0.0625, 0.0625, 0.0625, 0.25, 0.0625, 0.0625, 0.125, 0.0625, 0.0625, 0.0625, 1.0, 0.0625, 0.0625, 0.25, 0.0625, 0.0625, 0.5, 0.125, 0.25, 0.125, 0.5, 0.0625, 0.0625, 0.25, 0.0625, 0.0625, 1.0, 0.0625, 0.0625, 0.125, 0.0625, 0.0625, 0.0625, 0.25, 0.5, 0.25, 1.0, 0.125, 0.25, 0.125, 1.0, 0.125, 0.25, 0.125, 0.5, 0.0625, 0.0625, 0.0625, 0.25, 0.0625, 0.0625, 0.0625, 1.0, 0.0625, 0.0625, 0.0625, 0.25, 0.0625, 0.0625, 0.0625, 0.125, 0.0625, 0.0625, 0.0625, 0.5, 0.0625, 0.0625, 0.0625, 0.125, 0.0625, 0.06

In [9]:
# Smith-Waterman alignment between the two input sequences
def swm_alignment(string1, string2):
    match = 2
    mismatch = -2
    scoring = NucleotideScoringMatrix(match, mismatch)
    sw = LocalAlignment(scoring)#, gap_extension_penalty = -5) 
    
    alignment = sw.align(string1, string2)
    alignment_strings = alignment.dump()

    simi_score = alignment.matches / (alignment.mismatches + alignment.matches)
    
    return alignment_strings, simi_score


In [10]:
def filter_and_gather_patterns(allthepatterns, fullseq):
    pat = fullseq.split('-')
    print("num of pat in this seq:", len(pat))    
    for pattern in pat:
        # Get rid of the sequence shorter than 4; combine all alignment patterns into all pattern list
        if len(pattern) > 2:
            allthepatterns.append(pattern)
    return allthepatterns

In [11]:
# compare each pair of voices!
allthepatterns = []

for i in range(0, len(voices)-1):
    for j in range(i+1, len(voices)):
        alignment_strings, simi_score = swm_alignment(strings[i], strings[j])
        if simi_score < 0.25:
            print("ignoring voice pair:", i, j)
            # if two voices are very different, getting patterns from their alignment won't make sense
            continue
        # get the patterns from the first voice of this pair
        fullseq1 = alignment_strings[0]
        fullseq2 = alignment_strings[1]
        allthepatterns = filter_and_gather_patterns(allthepatterns, fullseq1)
        allthepatterns = filter_and_gather_patterns(allthepatterns, fullseq2)
        

Query:   2 IAJEIEEEKEEEKEIFAM---FFFF-----HFFFFGGMFFFFGGMFFFFGGMFGG----MGGGFFFFM-GGGFFFFMMMFGGGFFFFMMGGGFFFFDIIHCHCHHEICEIE--MGMFGGLFJMCIFFFFHFFFFGGMFFFFGGMFFFFGGMFGGFEKEMGGGFFFF---------HCH--FIAJEIEKFFFFHHHFEFFGGCLFGGGMFGE------CHCHCMMCHCHCMDE----K------------GGFFFFFFGEF-FGJFFFFFFKFFFFFFF--GJGGFFGG 238
           ||||||||||||||...|   ||||     |||||||||||||||||||||||||    |  |    | |||        |||      |||   |  |    ..|| |.|  .|  || |     ||||   .||.||||    ||| | |  |      .||||         |||  .|      ||||||||||||||||     .|      .|            |    |            ||||||||  | |  ||||||.|||||||  |  |.|||
Ref  :   2 IAJEIEEEKEEEKEGGMMGGGFFFFGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGDEIEM--G----MFGGG--------GGG------GGG---F--I----AJEI-EEEKEEG--GG-F-----FFFH---GGGGFFFF----FFF-G-F--F------FFFFFMFMCIMMIMHCHCHMI------FFFFHHHFEFFGGCLF-----IEIAJEIEEH------------EIEEEKEEEKEMFGGGGGGGFFFFFF--FIF--FFFFFFIFFFFFFFKFG--GMFGG 205

num of pat in this seq: 52
num of pat in this seq: 85
Query: 122 EKCE-IMMGGGFFFFGFFFFFFFMFF

In [12]:
allthepattern_count = dict(Counter(allthepatterns))
# elimiate the repeated strings

allthepatterns = list(set(allthepatterns))

In [47]:
len(allthepatterns)

60

In [49]:
def count_substringandparents(allthepatterns, allthepattern_count):
    
    issubstring = {}
    hassubstring = {}
    count_as_substring = {}
    count_as_parent = {}

    for i in range(0, len(allthepatterns)-1):
        for j in range(i+1, len(allthepatterns)):
            if allthepatterns[i] in allthepatterns[j]:
                parent = allthepatterns[j]
                kid = allthepatterns[i]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] = 1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1#times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 

            elif allthepatterns[j] in allthepatterns[i]:
                parent = allthepatterns[i]
                kid = allthepatterns[j]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] =1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1 #times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 #times

    return issubstring, hassubstring, count_as_substring, count_as_parent


In [50]:
issubstring, hassubstring, count_as_substring, count_as_parent = count_substringandparents(allthepatterns, allthepattern_count)

In [51]:
# rank the ones that are substrings of another detected pattern
sortis = dict(Counter(issubstring))
sorthas = dict(Counter(hassubstring))
print(len(sorthas))
print(len(sortis))

46
36


In [52]:
# This counts the times each pattern appeared after original segmentation plus the times they are in other patterns,
# plus the times other patterns showed up in them.

def combined_count(sorthas, sortis, count_as_parent, count_as_substring, allthepattern_count):
    refined_combined = {}

    for item in sorthas:
        if item in sortis:
            # the ones that has substring and also are the substrings
            refined_combined[item] = count_as_parent[item] + count_as_substring[item] + allthepattern_count[item]
        else:
            # the ones that has substrings
            refined_combined[item] = count_as_parent[item] + allthepattern_count[item]
    # the patterns that are substrings of other patterns but none of the others are substrings of it
    for item in sortis:
        if item not in refined_combined:
            refined_combined[item] = count_as_substring[item] + allthepattern_count[item]
        # otherwise is already counted
    
    return refined_combined

combined = combined_count(sorthas, sortis, count_as_parent, count_as_substring, allthepattern_count)

In [53]:
#dict(Counter(combined))

In [54]:
def filter_combined_patterns(combined):
    
    finalpattern = []
    for i in combined: 

        if len(i) < 3:
            # if the length is shorter than 4 notes, discard
            continue

        if combined[i] > 1:
            # keep the ones that has substring or is substring more than once
            finalpattern.append(i)

    finalpattern_combined = {}
        
    for thisone in finalpattern:
        finalpattern_combined[thisone] = combined[thisone]

    dict(Counter(finalpattern_combined))
        
    if len(finalpattern) > 30:
        # if more than 20 patterns, only take the top 20
        finalpattern_combined = dict(sorted(finalpattern_combined.items(), key = lambda x:-x[1], reverse = True)[-20:])
    
    return finalpattern_combined


In [55]:
# filter out the short ones
top20patterns_combined = filter_combined_patterns(combined)

In [56]:
print(dict(Counter(top20patterns_combined)))

{'AJEI': 10, 'HCH': 10, 'IMMGGGFFFFGFFFFFFFMFFFFHFFFFGGMFFFFGGMFFFFGGMFGG': 11, 'GMFGGGG': 11, 'MGMFGGLFJMCIFFFFHFFFFGGMFFFFGGMFFFFGGMFGGFEKEMGGGFFFF': 12, 'HFFFFGGMFFFFGGMFFFFGGMF': 12, 'AMFFFFHFFFFGGMFFFFGGMFFFFGGMFGGMGGGFFFFMGGGFFFF': 12, 'HFFFFGGMFFFFGGMFFFFGGMFGG': 13, 'FFFH': 13, 'IAJEIEEEKEEEKEGGMMGGGFFFFGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGDEIEM': 14, 'MFGGG': 14, 'GGFFFFJEKCEIMMGGGFFFFGFFFFFFFMFFFFHFFFFGGMFFFFGGMFFFFGGMFGGGGMF': 15, 'IAJEIEEEKEEEKEIFAMFFFFHFFFFGGMFFFFGGMFFFFGGMFGGMGGGFFFFMGGGFFFFMMMFGGGFFFFM': 16, 'GGGG': 16, 'GMFGG': 20, 'GGFF': 27, 'GGG': 39, 'FFGG': 39, 'FFFF': 79, 'FFF': 91}


In [57]:
def reduce_pattern_length(seq):
        # the function returns the longest substring that appeared at least once
        # if there is length 4 substring repeated 4 times and a length 7 substring repeated 2 times, we take the length 7
        best_performance = ""
        candidates = []
        for length in range(int(0.3*len(seq)), int(len(seq)*0.5)+1): 
            for start in range(0, len(seq)-length):
                # get all the substrings of this length within the string, save in candidates
                candidates.append(seq[start:start+length])
        count_can = {}
        for candidate in candidates:
                count_can[candidate] = seq.count(candidate)
                if count_can[candidate] > 1:
                    # if it is repeated more than once in the string
                    if len(candidate) > len(best_performance):
                        best_performance = candidate
                    elif len(candidate) == len(best_performance) and count_can[candidate] > count_can[best_performance]:
                        best_performance = candidate
        
        return best_performance

def check_reduction(item):
    success = False
    frequent_substr = reduce_pattern_length(item)
    if len(frequent_substr) > 4 and len(frequent_substr) > 0.3*len(item):
        print("extracted", frequent_substr, "from", item)
        # sucessfully reduced the pattern
        return frequent_substr
    return None

In [58]:
def reduce_length_of_each_pattern(onlyassubstring, top20patterns):
    
    beforereduction = {}
    # reducedfinalpat is the new list that contains all valid patterns after reduction plus the ones that does not need to be reduced
    reducedfinalpat = []

    for item in top20patterns:
        # check the ones that has substring, see if their length can be reduced
        if len(item) > 11 and item not in onlyassubstring:
            #if the extracted pattern is too long, find longest repeated pattern within it
            frequent_substr = check_reduction(item)
            if frequent_substr != None:
                if len(frequent_substr) > 23:
                    # second attempt, if a frequent substring is takend and it is still quite long
                    new_frequent_substr = check_reduction(frequent_substr)
                    if new_frequent_substr != None:
                        # if the second attempt is a success, take the twice reduced pattern
                        reducedfinalpat.append(new_frequent_substr)
                        beforereduction[new_frequent_substr] = item
                    else:
                        # if the second attempt failed, take the result of first reduction
                        reducedfinalpat.append(frequent_substr)
                        beforereduction[frequent_substr] = item
                else:
                    # if the extracted pattern is already not very long
                    reducedfinalpat.append(frequent_substr)
                    beforereduction[frequent_substr] = item
            else:
                # if the reduction failed, take the original
                reducedfinalpat.append(item)
        else:
            # otherwise keep it without reduction
            reducedfinalpat.append(item)
    
    return reducedfinalpat, beforereduction

In [59]:
onlyassubstring = set(issubstring) - set(hassubstring)
reducedfinalpat, beforereduction = reduce_length_of_each_pattern(onlyassubstring, list(set(top20patterns_combined)))

extracted FFFFGGMF from HFFFFGGMFFFFGGMFFFFGGMF
extracted FFFFGGMF from HFFFFGGMFFFFGGMFFFFGGMFGG


In [60]:
def find_pattern_pos(string, pattern, beatstrength):
    patpos_start = []
    patpos_end = []
    count = 0
    beatcount = 0
    for m in re.finditer(pattern, string):
        count +=1
        if beatstrength[m.start()] == 1: # or >= 0.5
            beatcount+=1
        patpos_start.append(m.start())
        patpos_end.append(m.end())
    return patpos_start, patpos_end, count, beatcount

In [61]:
def count_final_pattern(reducedfinalpat, strings, beatstrengths):

    count_pattern = {}
    refined_finalpat = []
    startonsets = []
    startpitches = []
    endonsets = []
    endpitches = []

    for pattern in reducedfinalpat:
        stringnum = 0
        beat_importance = 0
        for string in strings:
            # find the pattern in eachvoice
            patpos_start, patpos_end, count, beatcount = find_pattern_pos(string, pattern, beatstrengths[stringnum])
            beat_importance += beatcount
            if pattern not in count_pattern:
                count_pattern[pattern] = count
            else:
                count_pattern[pattern] += count
            if count != 0:
                print("Pattern", pattern, "appears in voice no.", stringnum+1, ", first pos starts at ", patpos_start[0], "end at", patpos_end[0])
            num = 0
            for startpos in patpos_start:
                startonsets.append(noteoffsets[stringnum][startpos])
                endonsets.append(noteoffsets[stringnum][patpos_end[num]])
                num+=1
            stringnum+=1
        if count_pattern[pattern] >= 2 and beat_importance > 0:
            # get rid of the ones that actually appeared less than 2 times in total
            refined_finalpat.append(pattern)
        else:
            if count_pattern[pattern] < 2:
                print("discard pattern", pattern, "because it only appered once in total.")
            elif beat_importance == 0:
                print("discard pattern", pattern, "because it never appeared at the start of a bar.")
                print("this pattern can be decoded as", decode_strings(pattern))
            
    return refined_finalpat, count_pattern, startonsets, endonsets


In [62]:
def decode_strings(string):
    
    decode_dia_intervals = {'A': '6D', 'B': '5D', 'C': '4D', 'D': '3D', 'E': '2D', 'F': '1D', 
                            'G': '1A', 'H': '2A', 'I': '3A', 'J': '4A', 'K': '5A', 'L': '6A', 'M': '0A'}
    
    decode_string = ""

    for letter in string:
        trans = decode_dia_intervals[letter]
        decode_string+=trans

    return decode_string

In [63]:
finalpat, count_pattern, startonsets,  endonsets = count_final_pattern(reducedfinalpat, strings, beatstrengths)

Pattern FFFF appears in voice no. 1 , first pos starts at  22 end at 26
Pattern FFFF appears in voice no. 2 , first pos starts at  19 end at 23
Pattern FFFF appears in voice no. 3 , first pos starts at  12 end at 16
Pattern MGMFGGLFJMCIFFFFHFFFFGGMFFFFGGMFFFFGGMFGGFEKEMGGGFFFF appears in voice no. 2 , first pos starts at  99 end at 152
discard pattern MGMFGGLFJMCIFFFFHFFFFGGMFFFFGGMFFFFGGMFGGFEKEMGGGFFFF because it only appered once in total.
Pattern GGFFFFJEKCEIMMGGGFFFFGFFFFFFFMFFFFHFFFFGGMFFFFGGMFFFFGGMFGGGGMF appears in voice no. 3 , first pos starts at  114 end at 177
discard pattern GGFFFFJEKCEIMMGGGFFFFGFFFFFFFMFFFFHFFFFGGMFFFFGGMFFFFGGMFGGGGMF because it only appered once in total.
Pattern AJEI appears in voice no. 1 , first pos starts at  2 end at 6
Pattern AJEI appears in voice no. 2 , first pos starts at  2 end at 6
Pattern AJEI appears in voice no. 3 , first pos starts at  2 end at 6
discard pattern AJEI because it never appeared at the start of a bar.
this pattern can be d

In [64]:
def decode_results(finalpat, count_pattern, combined, beforereduction):
    
    decoded_patterns = []
    decoded_patterns_weighed = {}
    decoded_patterns_count = {}
    decoded_patterns_combined = {}
    
    decode_dia_intervals = {'A': '6D', 'B': '5D', 'C': '4D', 'D': '3D', 'E': '2D', 'F': '1D', 
                            'G': '1A', 'H': '2A', 'I': '3A', 'J': '4A', 'K': '5A', 'L': '6A', 'M': '0A'}

    # decode all the candidates for patterns
    for pattern in finalpat:
        trans_pattern = ""
        for letter in pattern:
            trans = decode_dia_intervals[letter]
            trans_pattern += trans

        decoded_patterns_count[trans_pattern] = count_pattern[pattern]
        
        if pattern in combined:
            # some might be reduced to a shorter pattern
            decoded_patterns_combined[trans_pattern] = combined[pattern]
        else:
            decoded_patterns_combined[trans_pattern] = combined[beforereduction[pattern]]
        decoded_patterns.append(trans_pattern)
        
    return decoded_patterns, decoded_patterns_combined, decoded_patterns_count


Strings of intervals of each voice:

In [65]:
# Strings of intervals of each voice
decoded_strings = []
for string in strings:
    decoded = decode_strings(string)
    decoded_strings.append(decoded)
    print(decoded)
    print("")

2D3A6D4A2D3A2D2D2D5A2D2D2D5A2D1A1A0A0A1A1A1A1D1D1D1D1A1D1D1D1D2A1D1D1D1D1A1A0A1D1D1D1D1A1A0A1D1D1D1D1A1A0A1D1A1A3D2D3A2D0A1A0A1D1A1A1A1A1A1A1A1A1A1D3A6D4A2D3A2D2D2D5A2D2D1A1A1A1D1D1D1D2A1A1A1A1A1D1D1D1D1D1D1D1A1D1D1D1D1D1D1D0A1D0A4D3A0A0A3A0A2A4D2A4D2A0A3A1D1D1D1D2A2A2A1D2D1D1D1A1A4D6A1D3A2D3A6D4A2D3A2D2D2A2D3A2D2D2D5A2D2D2D5A2D0A1D1A1A1A1A1A1A1A1D1D1D1D1D1D1D3A1D1D1D1D1D1D1D3A1D1D1D1D1D1D1D5A1D1A1A0A1D1A1A0A1A4D2A1A

1D3A6D4A2D3A2D2D2D5A2D2D2D5A2D3A1D6D0A1D1D1D1D2A1D1D1D1D1A1A0A1D1D1D1D1A1A0A1D1D1D1D1A1A0A1D1A1A0A1A1A1A1D1D1D1D0A1A1A1A1D1D1D1D0A0A0A1D1A1A1A1D1D1D1D0A0A1A1A1A1D1D1D1D3D3A3A2A4D2A4D2A2A2D3A4D2D3A2D0A1A0A1D1A1A6A1D4A0A4D3A1D1D1D1D2A1D1D1D1D1A1A0A1D1D1D1D1A1A0A1D1D1D1D1A1A0A1D1A1A1D2D5A2D0A1A1A1A1D1D1D1D2A4D2A1D3A6D4A2D3A2D5A1D1D1D1D2A2A2A1D2D1D1D1A1A4D6A1D1A1A1A0A1D1A2D4D2A4D2A4D0A0A4D2A4D2A4D0A3D2D5A1A1A1D1D1D1D1D1D1A2D1D1D1A4A1D1D1D1D1D1D5A1D1D1D1D1D1D1D1A4A1A1A1D1D1A1A2A6D2A1A

2D3A6D4A2D3A2D2D2D5A6D0A1D1D1D1D2A1D1D1D1D1A1A0A1D1D1D1D1A1A0A1D1D1D1D1A1A0A1D1A1A0A1A1A1A1D

In [66]:
# Decode patterns
decoded_patterns, decoded_patterns_combined, decoded_patterns_count = decode_results(finalpat, count_pattern, combined, beforereduction)
    


for pattern in decoded_patterns:
        print("Pattern:", pattern)
        print("importance score:", decoded_patterns_combined[pattern],
              "occurrence:", decoded_patterns_count[pattern])


Pattern: 1D1D1D1D
importance score: 79 occurrence: 51
Pattern: 1A1A1A1A
importance score: 16 occurrence: 11
Pattern: 1D1D1D
importance score: 91 occurrence: 61
Pattern: 1A1A1D1D
importance score: 27 occurrence: 22
Pattern: 0A1D1A1A1A
importance score: 14 occurrence: 9
Pattern: 1D1D1A1A
importance score: 39 occurrence: 28
Pattern: 2A4D2A
importance score: 10 occurrence: 7
Pattern: 1A1A1A
importance score: 39 occurrence: 31


In [67]:
decoded_patterns

['1D1D1D1D',
 '1A1A1A1A',
 '1D1D1D',
 '1A1A1D1D',
 '0A1D1A1A1A',
 '1D1D1A1A',
 '2A4D2A',
 '1A1A1A']

In [68]:
# Get patterns from ground truth

dict_wordtonum = {"Unison": '0', "Second": '1', "Third": '2', "Fourth": '3', "Fifth": '4', "Sixth": '5', "Seventh": '6'}

groundtruthpatterns = []
print(glob.glob("jkupddtest/chopinmono/*"))
filenames = glob.glob("jkupddtest/chopinmono/B/*.csv")
for filename in filenames:
    print(filename)
    with open(filename, 'r') as file:
        csvreader = csv.reader(file)
        gts = []
        gtm = []
        pattern_gt = []
        for row in csvreader:
            gts.append(float(row[0]))
            gtm.append(float(row[1]))
        for i in gtm:
            pattern_gt.append(m21.note.Note(i))
        pattern_string = ""
        interval_string = ""
        for i in pattern_gt:
            pattern_string += str(i)
            
        for i in range(1, len(pattern_gt)):           
            gap = pattern_gt[i].pitch.diatonicNoteNum - pattern_gt[i-1].pitch.diatonicNoteNum
            if gap >= 0:
                direction = 'A'
            else:
                direction = 'D'
            m21_interval_directed = m21.interval.Interval(noteStart=pattern_gt[i-1], noteEnd=pattern_gt[i]).directedSimpleNiceName
            arr_diatonic = m21_interval_directed.split(" ")
            m21_generic = dict_wordtonum[arr_diatonic[-1]]
            m21_interval = m21_generic+direction
            
            interval_string+=m21_interval
            
        print(interval_string)

        groundtruthpatterns.append(interval_string)


['jkupddtest/chopinmono/A', 'jkupddtest/chopinmono/mazurka24-4.krn', 'jkupddtest/chopinmono/B']
jkupddtest/chopinmono/B/occ5.csv
2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A0A1A1D0A5D1A
jkupddtest/chopinmono/B/occ1.csv
2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A1A1D0A5D1A
jkupddtest/chopinmono/B/occ7.csv
2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A0A1A1D0A5D1A
jkupddtest/chopinmono/B/occ2.csv
2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A0A0A3D0A1D
jkupddtest/chopinmono/B/occ6.csv
2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A0A0A3D0A1D
jkupddtest/chopinmono/B/occ4.csv
2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A0A0A3D0A1D
jkupddtest/chopinmono/B/occ3.csv
2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A0A1A1D0A5D1A


In [69]:
set(groundtruthpatterns)

{'2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A0A1A1D0A5D1A',
 '2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A0A0A3D0A1D',
 '2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A1A1D0A5D1A'}

In [70]:
for pattern in groundtruthpatterns:
    count = 0
    for decodedstring in decoded_strings:
        count += decodedstring.count(pattern)
    if count == 0:
        print("pattern", pattern, "not in original score")
    elif count == 1:
        print("pattern", pattern, "only appeared once")
    else:
        print("pattern", pattern, "appeared", count, "times in total")

pattern 2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A0A1A1D0A5D1A not in original score
pattern 2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A1A1D0A5D1A not in original score
pattern 2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A0A1A1D0A5D1A not in original score
pattern 2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A0A0A3D0A1D not in original score
pattern 2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A0A0A3D0A1D not in original score
pattern 2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A0A0A0A3D0A1D not in original score
pattern 2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A0A1A1D0A5D1A not in original score


In [71]:
##CHECK IF THERE IS A MATCH

for i in groundtruthpatterns:
    if i in finalpat: #refined_finalpat:
        print("found a match:", i)

In [72]:
#TEST2


In [73]:
#groundtruthpath1 = "jkupddtest/gibbonsmono/silverswan.krn"
import pandas as pd
gtfiles = glob.glob("jkupddtest/chopinmono/B/midi/*.csv")
print(gtfiles)
gtstrings = []
for gtfile in gtfiles:
    print(gtfile)
    with open(gtfile, 'r') as file:
        df = pd.read_csv(file)
        #print(df)
        #for midi_note_num in df["midi_note_num"]:
        #    print(midi_note_num)
        count = 0
        gtstring = ""
        for diatonic in df["diatonic_interval"]:
            if count == 0:
                # ignore the first "interval" number which is always 0
                count+=1
                continue
            count+=1
            # just to format the input
            if diatonic < 0:
                dia = str(abs(diatonic)) + 'D'
            else:
                dia = str(abs(diatonic)) + 'A'
            gtstring += dia
        gtstrings.append(gtstring)
        print(gtstring)
            

['jkupddtest/chopinmono/B/midi/occ2.mid_part1.csv', 'jkupddtest/chopinmono/B/midi/occ6.mid_part1.csv', 'jkupddtest/chopinmono/B/midi/occ1.mid_part1.csv', 'jkupddtest/chopinmono/B/midi/occ7.mid_part1.csv', 'jkupddtest/chopinmono/B/midi/occ5.mid_part1.csv', 'jkupddtest/chopinmono/B/midi/occ4.mid_part1.csv', 'jkupddtest/chopinmono/B/midi/occ3.mid_part1.csv']
jkupddtest/chopinmono/B/midi/occ2.mid_part1.csv
2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A0A0A10D0A1D
jkupddtest/chopinmono/B/midi/occ6.mid_part1.csv
2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A0A0A10D0A1D
jkupddtest/chopinmono/B/midi/occ1.mid_part1.csv
2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A1A1D0A12D1A
jkupddtest/chopinmono/B/midi/occ7.mid_part1.csv
2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A7A1A1D0A12D1A
jkupddtest/chopinmono/B/midi/occ5.mid_part1.csv
2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A7A1A1D0A12D1A
jkupddtest/chopinmono/B/midi/occ4.mid_part1.csv
2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A0A

In [649]:
print(set(gtstrings))

{'2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A7A1A1D0A12D1A', '2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A1A1D0A12D1A', '2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A0A0A10D0A1D'}


In [650]:
for pattern in gtstrings:
    count = 0
    for decodedstring in decoded_strings:
        count += decodedstring.count(pattern)
    if count == 0:
        print("pattern", pattern, "not in original score")
    elif count == 1:
        print("pattern", pattern, "only appeared once")
    else:
        print("pattern", pattern, "appeared", count, "times in total")

pattern 2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A0A0A10D0A1D not in original score
pattern 2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A0A0A10D0A1D not in original score
pattern 2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A1A1D0A12D1A not in original score
pattern 2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A7A1A1D0A12D1A not in original score
pattern 2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A7A1A1D0A12D1A not in original score
pattern 2D2A2D0A1A0A0A4A3D3A3D0A0A0A0A4A2D2A2D0A1A0A7A0A0A10D0A1D not in original score
pattern 2D2A2D0A1A0A0A1A1D4A3D3A3D0A0A0A0A2A2D4A2D2A2D0A1A0A7A1A1D0A12D1A not in original score
